In [1]:
import os
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("solution_ex_08_v2").getOrCreate()

22/07/18 12:52:54 WARN Utils: Your hostname, certusr-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
22/07/18 12:52:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/07/18 12:52:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
inputPath = './data'
outputPath = './output_ex_08_v2'

print(os.path.exists(inputPath))
print(os.path.exists(outputPath))

True
False


In [9]:
df = spark.read.load(inputPath, format="csv", header=False, inferschema=True).withColumnRenamed('_c0', 'idSensor').withColumnRenamed('_c1', 'date').withColumnRenamed('_c2', 'PM10')
df.show()

+--------+----------+----+
|idSensor|      date|PM10|
+--------+----------+----+
|      s1|2016-01-01|20.5|
|      s2|2016-01-02|30.1|
|      s3|2016-01-03|27.4|
|      s1|2016-01-01|60.2|
|      s2|2016-01-02|20.4|
|      s3|2016-01-03|72.9|
|      s1|2016-01-03|55.5|
|      s2|2016-01-03|52.5|
|      s3|2016-01-02|32.5|
+--------+----------+----+



In [10]:
df.printSchema()

root
 |-- idSensor: string (nullable = true)
 |-- date: string (nullable = true)
 |-- PM10: double (nullable = true)



In [12]:
grp_df = df.groupBy("idSensor").max()
grp_df.show()

+--------+---------+
|idSensor|max(PM10)|
+--------+---------+
|      s2|     52.5|
|      s3|     72.9|
|      s1|     60.2|
+--------+---------+



In [14]:
# Define one group for each value of sensorId and compute the maximum value in each group and create another dataframe
sensorsMaxValueDF = df.groupBy("idSensor").agg({"PM10": "max"})
sensorsMaxValueDF.show()

+--------+---------+
|idSensor|max(PM10)|
+--------+---------+
|      s2|     52.5|
|      s3|     72.9|
|      s1|     60.2|
+--------+---------+



In [17]:
sensorsMaxValueDF.write.csv(outputPath)